# 余計なデータの削除

## tensorboard

In [1]:
# このステップ以下のファイルは削除する
min_step = 5000

In [2]:
import glob
path = '/root/user/work/logs/*/events*'
event_file_list = glob.glob(path)
event_file_list

['/root/user/work/logs/copy2copy_gumbel_latent16_epoch20_tau=0.5_k=0.001_x0=10000_TS=2019-Dec-05-10:01:34/events.out.tfevents.1575540096.0f950e7ca074.18825.0',
 '/root/user/work/logs/copy2copy_gumbel_latent16_epoch20_tau=0.1_k=0.001_x0=10000_TS=2019-Dec-05-10:01:18/events.out.tfevents.1575540080.0f950e7ca074.18714.0',
 '/root/user/work/logs/copy2copy_gumbel_latent200_epoch20_tau=0.1_k=0.001_x0=10000_TS=2019-Dec-05-10:01:32/events.out.tfevents.1575540094.0f950e7ca074.18743.0']

In [3]:
import pickle
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [4]:
def extract_scalar_from_event(path):
    event_acc = EventAccumulator(path, size_guidance={'scalars': 0})
    event_acc.Reload() # ログファイルのサイズによっては非常に時間がかかる
    target_key = ['step', 'value', 'wall_time']
    scalars = {}
    for tag in event_acc.Tags()['scalars']:
        events = event_acc.Scalars(tag)
        scalars[tag] = [{k: event.__getattribute__(k) for k in target_key} for event in events]
    return scalars

In [5]:
def extract_values(dict_list, key):
    return [_dict[key] for _dict in dict_list]

In [6]:
import os
import shutil

In [7]:
%%time
step_dict = []
for i, event_path in enumerate(event_file_list):
    print(f'\r{i}/{len(event_file_list)}: {event_path}', end='')
    event_dict = extract_scalar_from_event(event_path)
    event_dir = os.path.dirname(event_path)
    step = event_dict['TRAIN/ELBO'][-1]['step'] if event_dict else 0
    step_dict.append({
        'dir': event_dir,
        'path': event_path,
        'step': step,
    })

2/3: /root/user/work/logs/copy2copy_gumbel_latent200_epoch20_tau=0.1_k=0.001_x0=10000_TS=2019-Dec-05-10:01:32/events.out.tfevents.1575540094.0f950e7ca074.18743.0CPU times: user 4.17 s, sys: 620 ms, total: 4.79 s
Wall time: 4.22 s


In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(step_dict)

In [10]:
df

,dir,path,step
0,/root/user/work/logs/copy2copy_gumbel_latent16...,/root/user/work/logs/copy2copy_gumbel_latent16...,4685
1,/root/user/work/logs/copy2copy_gumbel_latent16...,/root/user/work/logs/copy2copy_gumbel_latent16...,4804
2,/root/user/work/logs/copy2copy_gumbel_latent20...,/root/user/work/logs/copy2copy_gumbel_latent20...,4548


In [11]:
# for _dir in df[df.step < min_step].dir.tolist():
#     print(_dir)
#     shutil.rmtree(_dir)